In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
import spotipy
import spotipy.util as util
import json
import numpy as np
from collections import defaultdict

CLIENT_ID = '06fdc33f688440e6bff40f6eb930f21c'
CLIENT_SECRET = 'f83c328423054a73a3daa9ae9045e538'
REDIRECT_URI = 'http://localhost/'
SPOTIFY_SCOPE = 'user-library-read playlist-modify-public'
username = 'james.hoke.peacock'

import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import seaborn as sns
%matplotlib inline

In [2]:
token = util.prompt_for_user_token(username, SPOTIFY_SCOPE, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)
sp = spotipy.Spotify(auth=token)
print token

BQCxJ8XvNONHvGtnbiqidrDeE0LIGbUEyS-kl9BUSbpqAumLhN04zZFNPVbL6y9bEif_Ie-FKt6tG2jrdukYhDDsgG0LD4xXZw9hPbOBgX4llz2PfUXaUGWn45FhEMhIXIV1vgiIZPYuJ5txVsaZOosDA9ANQBTUiIyAFKizMQdqXGkp6M1ZWVc9HEKpapeFK5yaBegblvWw8ACpHjiSaDGdXA-lU5KuBHVt


In [3]:
# Define a function for a bar plot
def barplot(x_data, y_data, error_data, x_label, y_label, title):

    _, ax = plt.subplots()
    # Draw bars, position them in the center of the tick mark on the x-axis
    ax.bar(x_data, y_data, color = '#539caf', align = 'center')
    # Draw error bars to show standard deviation, set ls to 'none'
    # to remove line between points
    ax.errorbar(x_data, y_data, yerr = error_data, color = '#297083', ls = 'none', lw = 2, capthick = 2)
    ax.set_ylabel(y_label, rotation='horizontal')
    ax.set_xlabel(x_label)
    ax.set_title(title)
    plt.xticks(rotation=45)


In [4]:
def mean_and_std(mat):
    # Assumes summing vertically. Horizontal dimension preserved
    npmat = np.array(mat)
    mean = np.mean(npmat, axis=0)
    std = np.std(npmat, axis=0)
    return mean, std
    

In [5]:
def to_key(k):
    return ["C","D♭","D","E♭","E","F","F#","G","A♭","A", "B♭","B"][k]

def to_camelot(k, reverse=False):
    
    if reverse and k.endswith("B"):
        return major[int(k[:-1])] + "Major"
    elif reverse:
        return minor[int(k[:-1])] + "Minor"
        
    k,m = k.split(" ")
    major = ["_", "B", "F#", "D♭", "A♭", "E♭", "B♭", "F", "C", "G", "D", "A", "E"]
    minor = ["_", "A♭", "E♭", "B♭", "F", "C", "G", "D", "A", "E", "B", "F#", "D♭"]        
    
    if m == "Major":
        return major.index(k), "B"
    else:
        return minor.index(k), "A"
    
def m_change(m):
    if m == "A":
        return "B"
    else:
        return "A"
    
def next_song_key(song_key, choice=0):
    k,m = to_camelot(song_key)
    
    cs = [(k+1,m), (k-1,m), (k,m_change(m)), (k+2, m), (k+3,m)]
    # JAWS mix is +7 or -5
    # Flip is across wheel +/- 6
    # pay attention is +3
    # most common is +1
    # also harmonic is -1
    # other common is +0 B->A
    # energy boost is +2 and change energy level (use sparingly)
    # other pay attention is +9
    
    new_k, new_m = cs[choice]
    
    return to_camelot(str(new_k)+new_m, True)
    
    
    


In [16]:
def hist_viz(song_data, features, n_bins=15):
    ppd = pd.DataFrame(data=song_data,columns=features)
    ppd.hist(bins=[i/float(n_bins) for i in range(n_bins+1)], color='steelblue', edgecolor='black', linewidth=1.0,
               xlabelsize=8, ylabelsize=8, grid=False, density=0, sharey=True) 
    plt.tight_layout(rect=(0, 0, 1.2, 1.2))


In [20]:
FEATURE_VECTOR = [
#     'acousticness',
    'danceability',
#     'duration_ms',
    'energy',
#     'instrumentalness',
    'key',
#     'liveness',
#     'loudness',
    'mode',
#     'speechiness',
    'tempo',
#     'time_signature',
    'valence'
]
def fetch_and_viz(song_uris, display_songs=False):
    track_features = sp.audio_features(song_uris)
    features = [u'energy', u'tempo', u'instrumentalness', u'danceability', u'key', u'valence', u'loudness', u'mode']
    features = FEATURE_VECTOR
    song_data = []
    if display_songs:
        print("Songs Featured")
        print([sp.track(track['uri']['name']) for track in track_features])
    for track in track_features:
        vals = {}
        for f in features:
            vals[f] = track[f]
        
        if 'tempo' in vals:
            vals['tempo'] = float((vals['tempo'] - 24)) / 204
        if 'key' in vals:
            vals['key'] /= float(12)
#         vals['loudness'] = float(vals['loudness'] + 10) / 11
        
        song_data.append([vals[f] for f in features]) # ensure ordering rather than using .values()
    hist_viz(song_data, features)
    
def mean_std_dev_vix(song_data):
    mean, std_dev = mean_and_std(song_data)
    barplot(x_data = features
        , y_data = mean
        , error_data = std_dev
        , x_label = 'Features'
        , y_label = 'Values'
        , title = 'Playlist Analysis')
    plt.ylim([0,1])


In [13]:
def vizualize_playlist(playlist_uri, display_songs=False):
    plid = playlist_uri.split(":")[-1]
    plist =  sp._get("playlists/%s" % (plid))
    print(plist['name'])
    track_results = sp._get("playlists/%s/tracks" % (plid), limit=100, offset=0,market='US')
    all_song_uris = [track[u'track'][u'uri'] for track in track_results['items']]
    fetch_and_viz(all_song_uris, display_songs)
    
def vizualize_song(song_uri):
    fetch_and_viz([song_uri])

In [9]:
# fetch_and_viz(['spotify:track:4BdGO1CaObRD4La9l5Zanz'])
# fetch_and_viz(['spotify:track:1ROBixGgXrYlcCcrBfxAoy'])
# Foster the people and Deadmau
# fetch_and_viz(['spotify:track:4BdGO1CaObRD4La9l5Zanz','spotify:track:1ROBixGgXrYlcCcrBfxAoy'], display_songs=True)

In [14]:
groove_theory_uri='spotify:user:spotify:playlist:37i9dQZF1DX1tW4VlEfDSS' #dance/disco
rap_caviar_uri = 'spotify:user:spotify:playlist:37i9dQZF1DX0XUsuxWHRQd' # rap/hip/hop
# pollen_uri = 'spotify:user:spotify:playlist:37i9dQZF1DWWBHeXOYZf74' # casual pop, all good songs
chill_n_bassy_uri = 'spotify:user:james.hoke.peacock:playlist:0Xl3RIy0jPlv3axAB1pMoR' # electronic
house_home_uri = 'spotify:user:heathrfeathr:playlist:3xwAMNQ6hHChEkG8469Iy1' # deep techno
# skrt_uri = 'spotify:user:miragefx:playlist:0JQJjjtY4GRaYgX1c8Qeo2' # casual pop
most_necessary_uri = 'spotify:user:spotify:playlist:37i9dQZF1DX2RxBh64BHjQ' # rap/hip/hop (new wave - lil pump, xxx)
this_is_tim_uri = 'spotify:user:spotify:playlist:37i9dQZF1DX1PQl5Hsjdb8' # country
this_is_metallica = 'spotify:user:spotify:playlist:37i9dQZF1DXbj9Ksq4BAdj' # rock
african_heat_uri = 'spotify:user:spotify:playlist:37i9dQZF1DWYkaDif7Ztbp' # afro

In [25]:
sativa_uri = 'spotify:user:12169714076:playlist:2BN9drdKaJWE8lYONYxgKz'
sunnier_uri = 'spotify:user:12169714076:playlist:7yJV6RL9lBqe427rvqCUid'
clouds_uri = 'spotify:user:12169714076:playlist:1gR0PzOwbLYwxECgtIvbty'
# vizualize_playlist(sativa_uri)
# vizualize_playlist(sunnier_uri)
# vizualize_playlist(clouds_uri)
get_genres_from_playlst(sativa_uri)
get_genres_from_playlst(sunnier_uri)
get_genres_from_playlst(clouds_uri)

pop 3.28571428571
electropop 2.0
r&b 1.57142857143
alternative r&b 1.42857142857
pop rap 1.42857142857
pop 0.571428571429
reggaeton 0.428571428571
australian indie 0.428571428571
latin 0.428571428571
irish pop 0.428571428571
vapor soul 1.42857142857
electropop 1.0
vapor twitch 0.857142857143
pop 0.714285714286
indietronica 0.714285714286


In [11]:
vizualize_playlist(most_necessary_uri)
# vizualize_playlist(rap_caviar_uri)
# vizualize_playlist(pollen_uri) # danceable, spread valence, spread energy, 
vizualize_playlist(skrt_uri)
vizualize_playlist(chill_n_bassy_uri)
vizualize_playlist(groove_theory_uri)

Most Necessary


KeyError: 'loudness'

In [ ]:
### OLD THOUGHTS (FROM MEAN/STD)
## Danceability, tempo & speechiness have narrowest std dev. Energy is next 
# instrumentalness,  liveness and valence have wide std dev for this playlist

In [ ]:

vizualize_playlist(house_home_uri) 
vizualize_playlist(groove_theory_uri)
# vizualize_playlist(pollen_uri)
vizualize_playlist('spotify:user:mightytreefolk:playlist:0RKqoL574g9rzdd0HZkfLA') # casual groove

In [24]:
def hist_values(seq):
    hist = defaultdict(int)
    for i in seq:
        hist[i] += 1

    return hist

def get_genres_from_playlst(playlist_uri, top_n=5):
    plid = playlist_uri.split(":")[-1]
    track_results = sp._get("playlists/%s/tracks" % (plid), limit=100, offset=0,market='US')
    all_artists = [track[u'track'][u'artists'] for track in track_results['items']]
    all_genres = []
    uri_dict = {}
    for artists in all_artists:
        for artist in artists:
            artist_full = sp.artist(artist[u'uri'])
            if artist_full[u'genres']:
                all_genres.extend(artist_full[u'genres'])
    top_genres = sorted(hist_values(all_genres).items(),key=lambda a: a[1], reverse=True)[:top_n]
    for k,v in top_genres:
        print k,float(v)/len(track_results)
#     return top_genres

In [ ]:
get_genres_from_playlst(rap_caviar_uri, top_n=10)
african_heat_uri = 'spotify:user:spotify:playlist:37i9dQZF1DWYkaDif7Ztbp' # afro
get_genres_from_playlst(african_heat_uri, top_n=10)

In [ ]:
get_genres_from_playlst(chill_n_bassy_uri, top_n=10)

In [ ]:
available_genre_seeds = ["acoustic", "afrobeat","alt-rock","alternative","ambient","anime","black-metal","bluegrass","blues","bossanova","brazil","breakbeat","british","cantopop","chicago-house","children","chill","classical","club","comedy","country","dance","dancehall","death-metal","deep-house","detroit-techno","disco","disney","drum-and-bass","dub","dubstep","edm","electro","electronic","emo","folk","forro","french","funk","garage","german","gospel","goth","grindcore","groove","grunge","guitar","happy","hard-rock","hardcore","hardstyle","heavy-metal","hip-hop","holidays","honky-tonk","house","idm","indian","indie","indie-pop","industrial","iranian","j-dance","j-idol","j-pop","j-rock","jazz","k-pop","kids","latin","latino","malay","mandopop","metal","metal-misc","metalcore","minimal-techno","movies","mpb","new-age","new-release","opera","pagode","party","philippines-opm","piano","pop","pop-film","post-dubstep","power-pop","progressive-house","psych-rock","punk","punk-rock","r-n-b","rainy-day","reggae","reggaeton","road-trip","rock","rock-n-roll","rockabilly","romance","sad","salsa","samba","sertanejo","show-tunes","singer-songwriter","ska","sleep","songwriter","soul","soundtracks","spanish","study","summer","swedish","synth-pop","tango","techno","trance","trip-hop","turkish","work-out","world-music"]

In [ ]:
def do_rec_filtering(seed_artists, seed_genres, seed_tracks, seed_playlist):
    
    sp.recommendations(self, seed_artists, seed_genres,seed_tracks, limit, country, **kwargs)

## Section 2 Visualization

In [ ]:
# Correlation Matrix Heatmap
f, ax = plt.subplots(figsize=(10, 6))
corr = wines.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Wine Attributes Correlation Heatmap', fontsize=14)

In [ ]:
## Line plot (best with 2 colors for different datasets, need a lot of similar examples)
# Scaling attribute values to avoid few outiers
cols = ['density', 'residual sugar', 'total sulfur dioxide', 'fixed acidity']
subset_df = wines[cols]

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

scaled_df = ss.fit_transform(subset_df)
scaled_df = pd.DataFrame(scaled_df, columns=cols)
final_df = pd.concat([scaled_df, wines['wine_type']], axis=1)
final_df.head()

# plot parallel coordinates
from pandas.plotting import parallel_coordinates
pc = parallel_coordinates(final_df, 'wine_type', color=('#FFE888', '#FF9999'))

In [ ]:
def analyze_region(): kms clustering on segments (timbre and pitch vectors) to separate into sound bytes if you will that are repeated - then you can display the timbre/pitch of these segments as well as play the via song playback at exact ms position
    
    save these segment_groups as compressed, synthesized information on database
    

In [ ]:
## FREE WORK SPACE
sp.artist('spotify:artist:4WUGQykLBGFfsl0Qjl6TDM') # the magician
sp.artist('spotify:artist:0DdDnziut7wOo6cAYWVZC5') # KREAM

In [ ]:
genre based segment clustering and replay to vizualize the sound of a a song but then also a genre? and then a playlist below a genre


how to process vector based data like timbre/pitch --> pca? clustering? timbre will probably be more indicative of genre because pitch is everywhere for all songs but for matching "notes" of songs the  
what is difference between single song viz and genre/playlist

In [ ]:
Song jumper like being able to programattically pick songs that would be good to mix into based on mixed in key logic/guide
Know that tempo, valence, energy & key will be important
It could suggest songs - java android app could live on phone - hear song and grab track id from shazam - spotify uri - features - suggest


In [ ]:
tf = sp.audio_features('spotify:track:339dwcSBa8oxVZxicvJG3c')[0]
t = sp.track('spotify:track:339dwcSBa8oxVZxicvJG3c')
def disp_key_info(tf):
    k = to_key(tf['key']) + [" Major", " Minor"][tf['mode']]
    t = str(round(tf['tempo']))
    e = str(round(tf['energy'] * 10, 1))
    print(k + " " + t + " " + e)
    
def disp_plist_info(playlist_uri):
    plid = playlist_uri.split(":")[-1]
    plist =  sp._get("playlists/%s" % (plid))
    print(plist['name'])
    track_results = sp._get("playlists/%s/tracks" % (plid), limit=100, offset=0,market='US')['items']
    c = 0
    for t in sp.audio_features([t['track']['uri'] for t in track_results]):
#         print(track_results[c]['track']['name'])
        disp_key_info(t)
        c += 1
    
mag_cle_uri = 'spotify:user:james.hoke.peacock:playlist:0KTNO7GW51S0kFE4jhjaEK'
this_chris_lake = 'spotify:user:spotify:playlist:37i9dQZF1DZ06evO3mGMY8'
disp_plist_info(this_chris_lake)

In [ ]:
def get_headers():
    return {"Accept":"application/json", "Content-Type":"application/json","Authorization": "Bearer %s" % token}

def recs(seed_artists=None, seed_genres=None,seed_tracks=None, limit=20, country=None, **kwargs):
    params = dict(limit=limit)
    if seed_artists:
        params['seed_artists'] = ','.join([sp._get_id('artist', a) for a in seed_artists])
    if seed_genres:
        params['seed_genres'] = ','.join(seed_genres)
    if seed_tracks:
        params['seed_tracks'] = ','.join([sp._get_id('track', t) for t in seed_tracks])
    if country:
        params['market'] = country
    
    print(params)

    for attribute in ["acousticness", "danceability", "duration_ms",
                      "energy", "instrumentalness", "key", "liveness",
                      "loudness", "mode", "popularity", "speechiness",
                      "tempo", "time_signature", "valence"]:
        for prefix in ["min_", "max_", "target_"]:
            param = prefix + attribute
            if param in kwargs:
                params[param] = kwargs[param]
    args = dict(params=params)
    headers = get_headers()
    return requests.get("https://api.spotify.com/v1/recommendations", headers=get_headers(), params=params).text

In [ ]:
tch = "1KpCi9BOfviCVhmpI4G2sY"
nicky = "5ChF3i92IPZHduM7jN3dpg"
oliver = "5nki7yRhxgM509M5ADlN1p"
# recs(seed_artists=[tch], limit=10, country="US")#, target_energy=6, max_energy=7.5, target_danceability=8, min_danceability=6)

recss = sp.recommendations(seed_artists=[oliver], limit=50, country="US", max_energy=5)#, target_danceability=8)#, min_danceability=6))
for r in recss['tracks']:
#     print(r)
    print(r['name'] + " - " + str([a['name'] for a in r['artists']]))
    ft = sp.audio_features([r['uri']])[0]
    if ft['energy'] < .9:
        disp_key_info(ft)
    

In [ ]:
import requests
print(requests.get("https://api.spotify.com/v1/recommendations?country=US&limit=20&seed_artists=1KpCi9BOfviCVhmpI4G2sY", headers=get_headers()).text)


def disp_recs_info(song_uris):
    track_results = sp.tracks(song_uris)['items']
    c = 0
    for t in sp.audio_features([t['track']['uri'] for t in track_results]):
#         print(track_results[c]['track']['name'])
        disp_key_info(t)
        c += 1
